#Trabajo Final Grupo Nº 2

Rafael Perez	rafaelperezctes@gmail.com
Olinca Ayala	olincayalan88@gmail.com
###@Created on Dec Sat 2 16:57:20 2023
###@author: Grupo Nº 2


# Modulo Vision

Este cuaderno realiza la descarga y análisis de secuencias nucleotídicas de integronas de Shewanella utilizando Biopython.


In [1]:
# @title Instalación de librerias
import subprocess
import ipywidgets as widgets
from IPython.display import display
import pkg_resources

def instalar_librerias():
    librerias = [
        "import_ipynb",
        "nbconvert",
        "ipywidgets",
        "biopython",
        "pandas",
        "numpy",
        "matplotlib",
        "tqdm",
        "requests",
        "gdown",
        "fasttree",
        "nglview",
    ]

    barra_progreso = widgets.IntProgress(
        value=0,
        min=0,
        max=len(librerias),
        bar_style='info',
        orientation='horizontal'
    )

    etiqueta = widgets.Label(value='Iniciando instalación...')
    display(widgets.VBox([etiqueta, barra_progreso]))

    # Instalación de fasttree y mafft con apt-get (solo ejecutar en Colab)
    subprocess.run(["apt-get", "install", "fasttree", "mafft", "ghostscript"])

    for libreria in librerias:
        try:
            pkg_resources.require(libreria)
            etiqueta.value = f"{libreria} ya está instalado."
        except pkg_resources.DistributionNotFound:
            etiqueta.value = f"Instalando: {libreria}..."
            subprocess.run(["pip", "install", libreria])
        barra_progreso.value += 1

    etiqueta.value = 'Instalación completada.'
    # importo todas las librerias
# Crear botón
boton_instalar = widgets.Button(description='Instalar Librerías')

# Crear área de salida
output = widgets.Output()
display(output)

# Enlazar el botón con la función de instalación
boton_instalar.on_click(lambda b: instalar_librerias())

# Mostrar botón
#display(boton_instalar)

#instalar_librerias()


Output()

In [2]:
def main():
  instalar_librerias()
#main()

In [3]:
lista=['AY509004.fasta', 'U12441.2.fasta']

In [5]:
import os
import threading
from concurrent.futures import ThreadPoolExecutor

# Función para buscar en la base de datos y analizar resultados
def buscarBD_y_Analizar(seq_record, archivo, parte):
    # Importaciones necesarias dentro de la función
    from Bio.Blast import NCBIWWW, NCBIXML
    from io import StringIO

    print(f"Corriendo BLASTX para {archivo}, parte {parte}...")
    fasta_string = f">{seq_record.id}\n{str(seq_record.seq)}"
    result_handle = NCBIWWW.qblast("blastx", "pdb", fasta_string)

    blast_results = result_handle.read()
    result_handle.close()

    string_result_handle = StringIO(blast_results)
    b_record = NCBIXML.read(string_result_handle)

    E_VALUE_THRESH = 0.1
    resultados = []

    # Recorrer y filtrar los resultados de BLAST
    for alignment in b_record.alignments[:10]:  # Limitar a los primeros 10 resultados
        for hsp in alignment.hsps:
            if hsp.expect < E_VALUE_THRESH:
                resultado = {
                    "Archivo": archivo,
                    "Parte": parte,
                    "Proteina ID": alignment.accession,
                    "E-valor": hsp.expect
                }
                resultados.append(resultado)
                break
    return resultados

# Función para dividir secuencias largas en segmentos
def dividir_secuencia(seq_record, max_longitud=100000):
    largo = len(seq_record.seq)
    for inicio in range(0, largo, max_longitud):
        fin = min(inicio + max_longitud, largo)
        segmento = seq_record.seq[inicio:fin]
        yield segmento

# Función para procesar cada archivo y realizar búsqueda BLAST
def blast_pdb(lista_archivos):
    from Bio import SeqIO
    from Bio.SeqRecord import SeqRecord

    todos_los_resultados = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = []
        for archivo in lista_archivos:
            secuencias = SeqIO.parse(archivo, "fasta")
            for secuencia in secuencias:
                parte = 1
                for segmento in dividir_secuencia(secuencia):
                    seg_record = SeqRecord(segmento, id=secuencia.id)
                    future = executor.submit(buscarBD_y_Analizar, seg_record, archivo, parte)
                    futures.append(future)
                    parte += 1

        for future in futures:
            resultado = future.result()
            if resultado:
                todos_los_resultados.extend(resultado)

    return todos_los_resultados

# Función para seleccionar un resultado y descargar el archivo correspondiente
def seleccionar_y_descargar_resultado(resultados):
    # Ordenar los resultados por el E-valor
    resultados_ordenados = sorted(resultados, key=lambda x: x['E-valor'])

    for idx, resultado in enumerate(resultados_ordenados, 1):
        print(f"{idx}. Archivo: {resultado['Archivo']}, Parte: {resultado['Parte']}, Proteina ID: {resultado['Proteina ID']}, E-valor: {resultado['E-valor']}")

    eleccion = int(input("Seleccione el número del resultado para descargar: "))
    if eleccion == 0:
        return  # O alguna otra función que quieras llamar
    resultado_elegido = resultados_ordenados[eleccion - 1]

    # Descargar archivo PDB
    from Bio.PDB import PDBList
    pdb_id = resultado_elegido['Proteina ID'][:4]  # Obtener las primeras 4 letras del ID de la proteína
    pdbl = PDBList()
    pdbl.download_pdb_files([pdb_id], obsolete=False, pdir="BD/PDB/", file_format="pdb", overwrite=True)

    # Cambiar la extensión del archivo de .ent a .pdb
    original_file = os.path.join("BD/PDB/", "pdb" + pdb_id.lower() + ".ent")
    new_file = os.path.join("BD/PDB/", pdb_id.lower() + ".pdb")
    if os.path.exists(original_file):
        os.rename(original_file, new_file)

# Las siguientes líneas pueden ser descomentadas para probar el script con archivos específicos
# lista_archivos = ['AY509004.fasta', 'U12441.2.fasta']
# resultados = blast_pdb(lista_archivos)
# seleccionar_y_descargar_resultado(resultados)


In [6]:
# Clase para visualizar estructuras de proteínas en archivos PDB
class PDBViewer:
    def __init__(self, pdb_directory="BD/PDB"):
        self.pdb_directory = pdb_directory
        self.ensure_directory_exists(pdb_directory)

    # Asegura que el directorio para almacenar archivos PDB exista
    def ensure_directory_exists(self, directory):
        if not os.path.exists(directory):
            os.makedirs(directory)

    # Lee un archivo PDB y retorna su estructura
    def read_pdb(self, structure_id, filename):
        from Bio.PDB import PDBParser  # Importación local
        try:
            parser = PDBParser(PERMISSIVE=1)
            return parser.get_structure(structure_id, filename)
        except Exception as e:
            print(f"Error al leer el archivo PDB: {e}")
            return None

    # Lista todos los archivos PDB disponibles en el directorio especificado
    def list_pdb_files(self):
        pdb_files = [f for f in os.listdir(self.pdb_directory) if f.endswith('.pdb')]
        for idx, file in enumerate(pdb_files, start=1):
            print(f"{idx}: {file}")
        return pdb_files

    # Permite al usuario seleccionar un archivo PDB para visualizar
    def select_pdb_file(self, pdb_files):
        try:
            choice = int(input("Ingrese el número del PDB a visualizar (0 para salir): "))
            if choice != 0:
                return pdb_files[choice - 1]
        except ValueError:
            print("Por favor, ingrese un número válido.")
        return None

    # Muestra la estructura de la proteína utilizando una visualización en 3D
    def display_structure(self, structure):
        import nglview as nv  # Importación local
        view = nv.show_biopython(structure)
        return view


def visualizador():
    viewer = PDBViewer()
    while True:
        print("\nMenú Principal")
        print("1. Listar archivos PDB")
        print("2. Visualizar archivo PDB")
        print("3. Salir")
        opcion = input("Seleccione una opción: ")

        if opcion == '1':
            viewer.list_pdb_files()
        elif opcion == '2':
            pdb_files = viewer.list_pdb_files()
            selected_file = viewer.select_pdb_file(pdb_files)
            if selected_file:
                structure_id = os.path.splitext(selected_file)[0]
                structure = viewer.read_pdb(structure_id, os.path.join(viewer.pdb_directory, selected_file))
                if structure:
                    from IPython.display import display  # Importación local para la función display
                    view = viewer.display_structure(structure)
                    display(view)  # Muestra la visualización inmediatamente
                    break  # Sale del bucle del menú principal
        elif opcion == '3':
            break
        else:
            print("Opción no válida, por favor intente de nuevo.")

#if __name__ == "__main__":
#    visualizador()

